In [ ]:
import itertools
from pathlib import Path
from typing import Optional

import gff_io
import hmmer
import iseq_prof_analysis as analysis
import plotly.express as px
from dna_features_viewer import GraphicFeature, GraphicRecord
from fasta_reader import read_fasta
from iseq_prof import pfam

In [ ]:
analysis.load_config(verbose=True)
EVALUE_THRSHOLD = 1e-10

# Depth 49, Hybrid consensus

In [ ]:
root = analysis.config.chlamydia.root_dir
hybrid_consensus = analysis.config.chlamydia.hybrid_consensus
output_dir = root / "output0.01"
hybrid = list(read_fasta(root / hybrid_consensus))
print(f"# {hybrid_consensus}")
print(f"Number of targets: {len(hybrid)}")
print(f"Target 2: >{hybrid[1].defline}")

In [ ]:
clans = pfam.Clans()
def normalize_clan_name(name: Optional[str]) -> str:
    if name is None:
        return "Unclassified"
    return name

## Prokka+HMMER3

In [ ]:
assembly_gffs = {}

for assembly_gff in gff_io.read_gff(output_dir / "prokka" / "assembly.gff"):
    ID = assembly_gff.attributes_asdict()["ID"]
    assembly_gffs[ID] = assembly_gff

features = []
colors = itertools.cycle(px.colors.qualitative.Plotly)
clan_colors = {}
for domtbl_row in hmmer.read_domtbl(output_dir / "assembly" / "domtblout.txt"):
    assembly_gff = assembly_gffs[domtbl_row.query.name]
    offset = int(assembly_gff.start) - 1

    alifrom = (domtbl_row.ali_coord.start - 1) * 3
    alito = domtbl_row.ali_coord.stop * 3

    # [start, end)
    start = offset + alifrom
    end = offset + alito
    
    profile_name = domtbl_row.target.name
    profile_clan = normalize_clan_name(clans.get(domtbl_row.target.accession))
    if profile_clan not in clan_colors:
        clan_colors[profile_clan] = next(colors)

    strand = int(assembly_gff.strand + "1")
    feature = GraphicFeature(start=start, end=end, strand=strand,
                             color=clan_colors[profile_clan],
                             label=profile_name)
    features.append(feature)

record = GraphicRecord(sequence_length=len(hybrid[1].sequence), features=features)
# ax = record.plot(figure_width=20)[0]
# ax.figure.savefig('prokka_on_hybrid_depth49_consensus.png', bbox_inches='tight')
record.plot(figure_width=20);

## iSeq (epsilon=0.01)

In [ ]:
output_dir = root / "output0.01"
features = []
for item in gff_io.read_gff(output_dir / "assembly" / "output.gff"):

    atts = item.attributes_asdict()
    if float(atts["E-value"]) > EVALUE_THRSHOLD:
        continue
        
    profile_name = atts["Profile_name"]
    profile_clan = normalize_clan_name(clans.get(atts["Profile_acc"]))
    if profile_clan not in clan_colors:
        clan_colors[profile_clan] = next(colors)

    start = int(item.start)
    end = int(item.end)
    strand = int(item.strand + "1")
    feature = GraphicFeature(start=start, end=end, strand=strand,
                             color=clan_colors[profile_clan],
                             label=profile_name)
    features.append(feature)

record = GraphicRecord(sequence_length=len(hybrid[1].sequence), features=features)
# ax = record.plot(figure_width=20)[0]
# ax.figure.savefig('iseq_on_hybrid_depth49_consensus.png', bbox_inches='tight')
record.plot(figure_width=20);

## iSeq (epsilon=0.001)

In [ ]:
output_dir = root / "output0.001"
features = []
for item in gff_io.read_gff(output_dir / "assembly" / "output.gff"):

    atts = item.attributes_asdict()
    if float(atts["E-value"]) > EVALUE_THRSHOLD:
        continue
        
    profile_name = atts["Profile_name"]
    profile_clan = normalize_clan_name(clans.get(atts["Profile_acc"]))
    if profile_clan not in clan_colors:
        clan_colors[profile_clan] = next(colors)

    start = int(item.start)
    end = int(item.end)
    strand = int(item.strand + "1")
    feature = GraphicFeature(start=start, end=end, strand=strand,
                             color=clan_colors[profile_clan],
                             label=profile_name)
    features.append(feature)

record = GraphicRecord(sequence_length=len(hybrid[1].sequence), features=features)
# ax = record.plot(figure_width=20)[0]
# ax.figure.savefig('iseq_on_hybrid_depth49_consensus.png', bbox_inches='tight')
record.plot(figure_width=20);